<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Snowflake - Basic usage and data querying

<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Snowflake/Snowflake_Basics_and_data_querying.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #snowflake #data #warehouse #naas_drivers #snippet

**Author:** [Mateusz Polakowski](https://www.linkedin.com/in/polakowski/)

This notebook shows basic usage of Snowflake driver.

Below you can find essential operations for setting up the environment and querying data that already exists in your data warehouse.

## Input

### Import library

In [1]:
import os
from naas_drivers import snowflake
from snowflake.connector.errors import ProgrammingError

### Setup Snowflake account

If you don't have your SF account, you can easily set up a [30-day trial account with $400 budget here](https://signup.snowflake.com/).

### Credentials

In [2]:
# Here environment variables are used to pass Snowflake credentials, 
# but it's okay to do it in a different manner

sf_username=os.environ['SNOWFLAKE_USER']
sf_password=os.environ['SNOWFLAKE_PASSWORD']
sf_account=os.environ['SNOWFLAKE_ACCOUNT']

## Model

### Connecting to your Snowflake account

In [3]:
snowflake.connect(
    username=sf_username,
    password=sf_password,
    account=sf_account
)

### Environment setup

In [4]:
snowflake.database.get_current() is None

True

In [5]:
snowflake.set_environment(
    warehouse='COMPUTE_WH',
    database='SNOWFLAKE_SAMPLE_DATA',
    schema='TPCH_SF100',
    role='ACCOUNTADMIN'
)

In [6]:
snowflake.get_environment()

{'role': 'ACCOUNTADMIN',
 'database': 'SNOWFLAKE_SAMPLE_DATA',
 'schema': 'TPCH_SF100',
 'warehouse': 'COMPUTE_WH'}

## Output

### Creating new database and schema

In [7]:
snowflake.database.create('NAAS', or_replace=True)

{'results': [('Database NAAS successfully created.',)],
 'description': [ResultMetadata(name='status', type_code=2, display_size=None, internal_size=16777216, precision=None, scale=None, is_nullable=True)],
 'statement': 'CREATE OR REPLACE DATABASE NAAS'}

In [8]:
snowflake.database.use('NAAS', silent=True)

In [9]:
snowflake.schema.create('INGESTION_SCHEMA', silent=True)

In [10]:
snowflake.schema.use('INGESTION_SCHEMA', True)

### Executing custom query with a cursor

In [11]:
snowflake.cursor.execute('SHOW SCHEMAS;').fetchall()

[(datetime.datetime(2022, 8, 6, 11, 54, 7, 118000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INFORMATION_SCHEMA',
  'N',
  'N',
  'NAAS',
  '',
  'Views describing the contents of schemas in this database',
  '',
  '1'),
 (datetime.datetime(2022, 8, 6, 11, 54, 6, 819000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'INGESTION_SCHEMA',
  'N',
  'Y',
  'NAAS',
  'ACCOUNTADMIN',
  '',
  '',
  '1'),
 (datetime.datetime(2022, 8, 6, 11, 54, 6, 421000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>),
  'PUBLIC',
  'N',
  'N',
  'NAAS',
  'ACCOUNTADMIN',
  '',
  '',
  '1')]

### Data querying - wrong schema

In [12]:
snowflake.get_environment()

{'role': 'ACCOUNTADMIN',
 'database': 'NAAS',
 'schema': 'INGESTION_SCHEMA',
 'warehouse': 'COMPUTE_WH'}

In [13]:
query = 'SELECT * FROM CUSTOMER;'

In [14]:
# Querying table that doesn't exist in NAAS/INGESTION_SCHEMA
try:
    results_1_not_working = snowflake.execute(query)
except ProgrammingError as pe:
    print('Something went wrong!')
    print(pe)

Something went wrong!
002003 (42S02): SQL compilation error:
Object 'CUSTOMER' does not exist or not authorized.


### Data querying - valid command run with session environment properly set up

In [15]:
snowflake.set_environment(
    database = 'SNOWFLAKE_SAMPLE_DATA',
    schema = 'TPCH_SF100'
)

In [16]:
results_1_working = snowflake.execute(query)
results_1_working

{'results': [(14124577,
   'Customer#014124577',
   'ZwI7ggkE1pOA39,1KTdz P1Jj6,g',
   16,
   '26-686-193-6704',
   Decimal('8596.72'),
   'BUILDING',
   'ular packages use blithely express instructions? carefully unusual sheaves are slyly. iron'),
  (14124578,
   'Customer#014124578',
   '3j2DHNdlO55ek8GXkVQkJXcDpk5swcztQJqT5x',
   24,
   '34-617-623-6475',
   Decimal('9167.74'),
   'HOUSEHOLD',
   'ic platelets sleep. idly final '),
  (14124579,
   'Customer#014124579',
   'USU73Rs2KgX7zHLbZTW2rYcJYNTldL5NMJ,d',
   0,
   '10-247-845-8540',
   Decimal('3343.39'),
   'BUILDING',
   'quickly regular dependencies boost according to the blithely ironic ideas. quickly special foxes cajole carefull'),
  (14124580,
   'Customer#014124580',
   'qjVhfUHXADJXKC5hsLw9A4dhUs,F,kRUSNso8PRO',
   10,
   '20-799-683-2862',
   Decimal('2263.09'),
   'HOUSEHOLD',
   'g to the carefully regular instructions dazzle among the ironic, slow theodolites. furiously bold packages haggle.'),
  (14124581,
   'Cu

### Data querying - another valid command run

In [17]:
results_2 = snowflake.execute(query, n=100)

print(f"Rows returned: {len(results_2['results'])}")
results_2['results'][:2]

Rows returned: 100


[(14124577,
  'Customer#014124577',
  'ZwI7ggkE1pOA39,1KTdz P1Jj6,g',
  16,
  '26-686-193-6704',
  Decimal('8596.72'),
  'BUILDING',
  'ular packages use blithely express instructions? carefully unusual sheaves are slyly. iron'),
 (14124578,
  'Customer#014124578',
  '3j2DHNdlO55ek8GXkVQkJXcDpk5swcztQJqT5x',
  24,
  '34-617-623-6475',
  Decimal('9167.74'),
  'HOUSEHOLD',
  'ic platelets sleep. idly final ')]

### Data querying - mapping results to Pandas DataFrame

In [18]:
results_pandas = snowflake.query_pd(query, n=100)

print(f'Rows returned: {len(results_pandas)}')
results_pandas.head(2)

Rows returned: 100


,C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
0,14124577,Customer#014124577,"ZwI7ggkE1pOA39,1KTdz P1Jj6,g",16,26-686-193-6704,8596.72,BUILDING,ular packages use blithely express instruction...
1,14124578,Customer#014124578,3j2DHNdlO55ek8GXkVQkJXcDpk5swcztQJqT5x,24,34-617-623-6475,9167.74,HOUSEHOLD,ic platelets sleep. idly final


## Extra

### Objects: `cursor` and `connection`

Both provided by Snowflake connector, that allow to execute any functionality possible with the original connector.

In [19]:
snowflake.cursor.execute('SELECT CURRENT_WAREHOUSE()').fetchall()

[('COMPUTE_WH',)]

In [20]:
snowflake.connection.database

'SNOWFLAKE_SAMPLE_DATA'